In [2]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")

Main.FinancialBVAR

In [20]:
data_df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
first(data_df,20)

Row,Date,10Y_Yeld,10Y_Price,Stock_Price,Dividend,E,CAPE,T-Bill,CPI,Dividend_Yeld
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1951-12-01,0.02513,104.098,24.19,1.41333,2.44,12.5271,1.73,26.5,0.0584262
2,1952-01-01,0.02463,104.771,23.75,1.41667,2.42667,12.3641,1.57,26.3,0.0596493
3,1952-02-01,0.02617,103.585,23.81,1.42,2.41333,12.3623,1.54,26.3,0.0596388
4,1952-03-01,0.02505,104.824,23.74,1.43,2.4,12.2427,1.59,26.4,0.0602359
5,1952-04-01,0.02422,105.806,23.73,1.44,2.38,12.2005,1.57,26.4,0.0606827
6,1952-05-01,0.02384,106.373,24.38,1.45,2.36,12.4479,1.67,26.5,0.059475
7,1952-06-01,0.02415,106.295,25.08,1.45,2.34,12.6691,1.7,26.7,0.057815
8,1952-07-01,0.02466,106.034,25.18,1.45,2.34667,12.6784,1.81,26.7,0.0575854
9,1952-08-01,0.02552,105.458,24.78,1.45,2.35333,12.4347,1.83,26.7,0.0585149


In [21]:
last(data_df,20)

Row,Date,10Y_Yeld,10Y_Price,Stock_Price,Dividend,E,CAPE,T-Bill,CPI,Dividend_Yeld
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2022-12-01,0.0388,4337.28,3960.66,67.35,172.75,28.3348,4.25,299.17,0.0170048
2,2023-01-01,0.0352,4481.05,4079.68,67.78,173.557,28.9198,4.54,300.84,0.016614
3,2023-02-01,0.0392,4348.09,3968.56,68.21,174.363,27.9381,4.65,301.836,0.0171876
4,2023-03-01,0.0348,4521.58,4121.47,68.3767,175.17,28.7648,4.69,303.363,0.0165904
5,2023-04-01,0.0344,4549.79,4146.17,68.5433,177.117,28.7621,4.92,304.127,0.0165317
6,2023-05-01,0.0364,4487.65,4345.37,68.71,179.063,29.94,5.14,305.109,0.0158122
7,2023-06-01,0.0381,4438.75,4508.08,68.911,181.01,30.8917,5.16,305.691,0.0152861
8,2023-07-01,0.0397,4395.09,4457.36,69.1121,182.09,30.3047,5.25,307.026,0.0155052
9,2023-08-01,0.0409,4366.98,4409.09,69.3131,183.17,29.7997,5.3,307.789,0.0157205


In [4]:
columns = [:Date, :short_us, :real_us, :term_us]
data_df = DataFrame(XLSX.readtable("shiller_data.xlsx", "Data"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
macro_data = TimeArray(data_df; timestamp = :Date)

ArgumentError: ArgumentError: column name :Date not found in the data frame